# inference

In [ ]:
!python detect.py --weights yolov3.pt --conf 0.25 --img-size 640 --source data/images/w.jpg --device=1 --project 'runs/detect' --name 'exp' --exist-ok

# 检测结果存入 runs/detect/exp, 打上 --exist-ok 允许覆盖

In [ ]:
import matplotlib.pyplot as plt
import cv2

src_img = cv2.imread('runs/detect/exp/w.jpg')
src_img = cv2.cvtColor(src_img,cv2.COLOR_BGR2RGB)
plt.imshow(src_img)
plt.show() 

# test&val

In [ ]:
!python test.py --weights yolov3.pt --data coco.yaml --batch-size 32 --img-size 640 --iou-thres 0.65 --task 'val' --device 1 --save-json --project 'runs/val' --name 'exp' --exist-ok

# 若val集是coco，建议 --save-json，以便调用pycocotools生成json文件，后期可以上传服务器打榜
# 结果存入 runs/val/exp ，允许覆盖

# train

In [ ]:
# tensorboard配置要指定host ip
%load_ext tensorboard
%tensorboard --logdir runs/train --host=10.13.23.46

In [ ]:
# wandb disable或者卸载，否则会产生对话要求login等，在jupyter中无法输入，只能在bash中
# 需要将训练数据上传至云端，不推荐，还是tensorboard

In [2]:
# yolov3-tiny on VOC
!python train.py --weights '' --cfg yolov3-tiny.yaml --img 640 --batch 16 --epochs 600 --data voc.yaml --project 'runs/train' --name 'exp_voc_3x' --rect --nosave --device 1 --exist-ok
# --weights '' --cfg yolov3.yaml 以yolov3的结构train from the scratch‘
# --weights yolov3.pt 则从预训练的pt文件开始
# --data 指定数据集
# --nosave 仅仅存最后的checkpoint
# --cache 会将数据缓存到内存里，注意炸满



github: ⚠️ WARNING: code is out of date by 1 commmit. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov3' to download latest.
YOLOv3 v9.1-3-g9f4e853 torch 1.7.1 CUDA:1 (TITAN RTX, 24189.0MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./models/yolov3-tiny.yaml', data='./data/voc.yaml', device='1', epochs=36, evolve=False, exist_ok=True, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp_voc_3x', noautoanchor=False, nosave=True, notest=False, project='runs/train', quad=False, rect=True, resume=False, save_dir='runs/train/exp_voc_3x', single_cls=False, sync_bn=False, total_batch_size=16, weights='', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv3 logging with 'pip install wandb' (recommended)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
hyperparameters: lr0=

     11/35     2.02G   0.05676   0.03369   0.03277    0.1232         7       256
               Class      Images     Targets           P           R      mAP@.5
                 all    4.95e+03     1.2e+04        0.41      0.0935       0.175      0.0538

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
     12/35     2.02G   0.05632   0.03348    0.0319    0.1217         7       256
               Class      Images     Targets           P           R      mAP@.5
                 all    4.95e+03     1.2e+04       0.358       0.127        0.19      0.0607

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
     13/35     2.02G   0.05555   0.03344   0.03068    0.1197         7       256
               Class      Images     Targets           P           R      mAP@.5
                 all    4.95e+03     1.2e+04       0.382        0.14        0.18      0.0535

     Epoch   gpu_mem       box       obj       cls     total   targets

In [ ]:
# yolov3-tiny on coco
!python train.py --weights '' --cfg yolov3-tiny.yaml --img 640 --batch 16 --epochs 600 --data coco.yaml --project 'runs/train' --name 'single_yolov3_tiny_coco_600eps' --rect --device 1 --exist-ok

# 采用了rectangle training

In [3]:
# DDP

# from the scratch
!python -m torch.distributed.launch --nproc_per_node=2 train.py --weights '' --cfg yolov3.yaml --img 640 --batch 32 --epochs 600 --data coco.yaml --project 'runs/train' --name 'distri_yolov3_coco_600eps' --sync-bn --exist-ok --workers 8

# continue
#!python -m torch.distributed.launch --nproc_per_node=2 train.py --weights 'runs/train/distri_exp_coco_3x/weights/last.pt' --img 640 --batch 32 --epochs 24 --data coco.yaml --project 'runs/train' --name 'distri_exp_coco_2x_continue' --nosave --sync-bn --exist-ok
# 注意这里的batch_size是全局batch，每个进程每个卡的batch_size=global bathcsize//GPUs
# DDP不匹配rect train




*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
github: ⚠️ WARNING: code is out of date by 1 commmit. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov3' to download latest.
YOLOv3 v9.1-3-g9f4e853 torch 1.7.1 CUDA:0 (TITAN RTX, 24190.3125MB)
                                   CUDA:1 (TITAN RTX, 24189.0MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./models/yolov3-tiny.yaml', data='./data/coco.yaml', device='', epochs=300, evolve=False, exist_ok=True, global_rank=0, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=0, log_artifacts=False, log_imgs=16, multi_scale=False, name='distri_yolov3_tiny_coco_300eps', noautoanchor=False, nosave=True, notest=False, pro

     9/299      4.9G   0.06208    0.1001   0.03945    0.2016        96       640
               Class      Images     Targets           P           R      mAP@.5
                 all       5e+03    3.63e+04       0.321       0.218       0.196      0.0838

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
    10/299      4.9G   0.06193    0.0996   0.03897    0.2005        92       640
               Class      Images     Targets           P           R      mAP@.5
                 all       5e+03    3.63e+04       0.326       0.222       0.199      0.0859

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
    11/299      4.9G    0.0616   0.09932   0.03853    0.1994        77       640
               Class      Images     Targets           P           R      mAP@.5
                 all       5e+03    3.63e+04       0.332       0.225       0.203      0.0879

     Epoch   gpu_mem       box       obj       cls     total   targets